In [ ]:
from PIL import Image, ImageOps
from IPython.display import display
import random
import json
import os.path


In [ ]:
### Adapted from https://github.com/vortextemporum/BoringBananasCompany/tree/master/image_generation
### Step 1: gather files

# Example of our backgrounds, use this format to fill in rest
background = ["Blue","Lilac","Orange","Pink","Purple","Yellow","Green","Greenburst"]
backgroundWeights = [27,20,18,12,10,7,5,1]
backgroundFiles = {"Blue":"blue.png","Lilac":"lilac.png","Orange":"orange.png","Pink":"pink.png","Purple":"purple.png","Yellow":"yellow.png","Green":"green.png","Greenburst":"greenburst.png"}

body = []
bodyWeights = []
bodyFiles = {}

face = []
faceWeights = []
faceFiles = {}

shirt = []
shirtWeights = []
shirtFiles = {}

headware = []
headwareWeights = []
headwareFiles = {}





In [ ]:
### Step 2: Generate traits, Total Souls = 9999 

TOTAL_SOULS = 9999

traits = []

def createCombo():
    
    trait = {}

    trait["Shirt"] = random.choices(shirt, shirtWeights)[0]
    trait["Background"] = random.choices(background, backgroundWeights)[0]
    trait["Body"] = random.choices(body, bodyWeights)[0]
    trait["Face"] = random.choices(face, faceWeights)[0]
    trait["Headware"] = random.choices(headware, headwareWeights)[0]
    
    
    if trait in traits:
        return createCombo()
    else:
        return trait
    
for i in range(TOTAL_SOULS):
    
    newtraitcombo = createCombo()
    
    traits.append(newtraitcombo)
    


In [ ]:
### Step 3: Check if there are any duplicates
def allUnique(x):
    seen = list()
    return not any(i in seen or seen.append(i) for i in x)

print(allUnique(traits))

In [ ]:
### Step 4: Add token Id to traits list

i = 0
for item in traits:
    item["tokenId"] = i
    i = i + 1

In [ ]:
### Step 5: Generate Images
    
for item in traits:

    im1 = Image.open(f'{backgroundFiles[item["Background"]]}').convert('RGBA')
    im2 = Image.open(f'{bodyFiles[item["Body"]]}').convert('RGBA')
    im3 = Image.open(f'{faceFiles[item["Face"]]}').convert('RGBA')
    im4 = Image.open(f'{shirtFiles[item["Shirt"]]}').convert('RGBA')
    im5 = Image.open(f'{headwareFiles[item["Headware"]]}').convert('RGBA')


    # Create each composite
    com1 = Image.alpha_composite(im1, im2)
    com2 = Image.alpha_composite(com1, im3)
    com3 = Image.alpha_composite(com2, im4)
    com4 = Image.alpha_composite(com3, im5)

    # Convert to RGB
    rgb_im = com4.convert('RGB')

    # Save image
    file_name = str(item["tokenId"]) + ".png"
    rgb_im.save("PATH/TO/OUTPUT_FOLDER/" + file_name)
    print(f'{str(item["tokenId"])} done')
    
    

In [ ]:
### Step 6: Build Json metadata files for opensea metadata standards and save into a folder

i = 0

for item in traits:
  metadata = {
      "image":"ipfs://PLACEHOLDER_IMAGE",
      "attributes":[
        {"trait_type":"Background", "value":item['Background']},
        {"trait_type":"Body", "value":item['Body']},
        {"trait_type":"Headware","value":item['Headware']},
        {"trait_type":"Face","value":item['Face']},
        {"trait_type":"Shirt","value":item['Shirt']}]
  }

  file_name = "PATH/TO/FOLDER/{}".format(i)
  with open(file_name, 'w') as outfile:
    json.dump(metadata, outfile, indent=4)
  i = i + 1

In [ ]:
### Step 7: Write one metadata file

file_name = "PATH/TO/JSON/FILE"
with open(file_name, 'w') as outfile:
    json.dump(traits, outfile, indent=4)

In [ ]:
### Step 8: Get trait counts

backgroundCounts = {}
for item in background:
    backgroundCounts[item] = 0

bodyCounts = {}
for item in body:
    bodyCounts[item] = 0

headwareCounts = {}
for item in headware:
    headwareCounts[item] = 0

shirtCounts = {}
for item in shirt:
    shirtCounts[item] = 0
   
faceCounts = {}
for item in face:
    faceCounts[item] = 0

allCounts = {};

for soul in traits:
    backgroundCounts[soul["Background"]] += 1
    bodyCounts[soul["Body"]] += 1
    headwareCounts[soul["Headware"]] += 1
    shirtCounts[soul["Shirt"]] += 1
    faceCounts[soul["Face"]] += 1

allCounts["Background"] = backgroundCounts
allCounts["Body"] = bodyCounts
allCounts["Headware"] = headwareCounts
allCounts["Shirt"] = shirtCounts
allCounts["Face"] = faceCounts
    
print("Background:", backgroundCounts)
print("Body:", bodyCounts)
print("Headware:", headwareCounts)
print("Shirt:", shirtCounts)
print("face:", faceCounts)

file_name = "PATH/TO/JSON/FILE"
with open(file_name, 'w') as outfile:
    json.dump(allCounts, outfile, indent=4)